In [59]:
import numpy as np
import sympy as sp
from IPython.display import display, Math



# Rescaling the energy

$$
E(y):=
\underbrace{\frac{-1}{2 Eh}\int_\Omega |\nabla^2 v|^2 - \nu [v, v]}_{\text{membrane}} +
\underbrace{\frac{D^*}{2}c_\nu\int_\Omega |\Delta w|^2 - (1-\nu) [w, w]}_{\text{bending}} +
\underbrace{\frac{1}{2 }\int_\Omega J^T \nabla^2 v J: \nabla w\otimes \nabla w}_{\text{coupling}} 
- \underbrace{\int_\Omega p w }_{\text{transv. pressure}} 
+ \underbrace{ \sum_i\int_\Omega s_i \delta_{x_i} v}_{\text{disclinations}} 
$$

where $E$ is the 3d Young's modulus and $D^*c_\nu:=D=\frac{E h^3}{12(1-\nu^2)}$ is the bending modulus, $h$ is the thickness of the plate

In [70]:
symbols = {str(key): sp.symbols(key) for key in ["v0", "w0", "E", "h", "D", "x0", "p0"]}

## Parameters
$v_0, w_0, E, h, D, x_0, p_0$

of which $v_0, w_0, x_0$ are free.

In [71]:
symbols

{'v0': v0, 'w0': w0, 'E': E, 'h': h, 'D': D, 'x0': x0, 'p0': p0}

In [72]:
list(symbols.keys())

['v0', 'w0', 'E', 'h', 'D', 'x0', 'p0']

In [73]:
v0, w0, E, h, D, x0, p0 = sp.symbols("v0 w0 E h D^* x0 p0")
R = sp.symbols("R")

In [78]:
coeff_list = {'membrane': v0**2 / (E * h * x0**2),
              'bending': D * w0**2 / x0**2,
              'coupling': v0 * w0 **2 / x0**2,
              'load_transverse': p0 * x0 **2 * w0,
              'load_disclinations': v0}


# E 

In [69]:
coeff_list

for key, value in coeff_list.items():
    latex_str = sp.latex(value)
    display(f"{key}", Math(f"{latex_str}"))

'membrane'

<IPython.core.display.Math object>

'bending'

<IPython.core.display.Math object>

'coupling'

<IPython.core.display.Math object>

'load_transverse'

<IPython.core.display.Math object>

'load_disclinations'

<IPython.core.display.Math object>

In [75]:

def apply_conditions(coeff_list):
    """Apply the conditions and solve for the parameters."""
    # Set x0 to 1
    x0_value = R
    D_value = E * h**3
    # E: Young's modulus (3D) 
    
    # Substitute x0 and D into the coefficients
    coeff_list_substituted = {key: coeff.subs({x0: x0_value, D: D_value}) 
                              for key, coeff in coeff_list.items()}
        
    return coeff_list_substituted

def rescale_by_membrane(coeff_list):
    """Rescale the coefficients by the membrane coefficient."""
    membrane_coeff = coeff_list['membrane']
    rescaled_coefficients = {key: sp.simplify(coeff / membrane_coeff) for key, coeff in coeff_list.items()}
    return rescaled_coefficients


def scaling(coeff_list):
    """Apply the conditions and solve for the parameters."""
    # Set x0 to 1
    x0_value = 1
    
    # Substitute x0 and D into the coefficients
    coeff_list_substituted = {key: coeff.subs({x0: R}) for key, coeff in coeff_list.items()}
    
    # Setting membrane, bending, and coupling coefficients to 1
    # membrane_condition = sp.Eq(coeff_list_substituted['membrane'], 1)
    coupling_condition = sp.Eq(coeff_list['coupling'], 1)
    bending_condition = sp.Eq(coeff_list['bending'], 1)
    
    # Solve for v0, w0
    v0_scale = sp.solve(bending_condition, v0)[0]
    w0_scale = sp.solve(coupling_condition.subs(v0, v0_scale), w0)[0]
    
    return v0_scale, w0_scale, coeff_list_substituted

In [77]:

coeff_list_substituted = apply_conditions(coeff_list)
coeff_list_substituted

for key, value in coeff_list_substituted.items():
    latex_str = sp.latex(value)
    display(f"{key}", Math(f"{latex_str}"))

'membrane'

<IPython.core.display.Math object>

'bending'

<IPython.core.display.Math object>

'coupling'

<IPython.core.display.Math object>

'load_transverse'

<IPython.core.display.Math object>

'load_disclinations'

<IPython.core.display.Math object>

# Rescaling the energy

$$
E(y):=
\frac{v_{0}^{2}}{E R^{2} h}\underbrace{\frac{-1}{2}\int_\Omega |\nabla^2 v|^2 - \nu [v, v]}_{\text{membrane}} +
\frac{E h^{3} w_{0}^{2}}{R^{2}}\underbrace{\frac{1}{2}c_\nu\int_\Omega |\Delta w|^2 - (1-\nu) [w, w]}_{\text{bending}} +
\frac{v_{0} w_{0}^{2}}{R^{2}}\underbrace{\frac{1}{2 }\int_\Omega J^T \nabla^2 v J: \nabla w\otimes \nabla w}_{\text{coupling}} 
- R^{2} p_{0} w_{0}\underbrace{\int_\Omega p w }_{\text{transv. pressure}} 
+ v_{0}\underbrace{ \sum_i\int_\Omega s_i \delta_{x_i} v}_{\text{disclinations}} 
$$

where $E$ is the 3d Young's modulus and $D^*c_\nu:=D=\frac{E h^3}{12(1-\nu^2)}$ is the bending modulus, $h$ is the thickness of the plate

In [79]:
rescaled_coefficients = rescale_by_membrane(coeff_list_substituted)
rescaled_coefficients

{'membrane': 1,
 'bending': E**2*h**4*w0**2/v0**2,
 'coupling': E*h*w0**2/v0,
 'load_transverse': E*R**4*h*p0*w0/v0**2,
 'load_disclinations': E*R**2*h/v0}

In [81]:

# Pretty print the rescaled coefficients
print("Rescaled coefficients by the membrane coefficient:")

for key, value in rescaled_coefficients.items():
    latex_str = sp.latex(value)
    display(f"{key}", Math(f"{latex_str}"))

Rescaled coefficients by the membrane coefficient:


'membrane'

<IPython.core.display.Math object>

'bending'

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

'load_transverse'

<IPython.core.display.Math object>

'load_disclinations'

<IPython.core.display.Math object>

In [82]:
coupling_condition = sp.Eq(rescaled_coefficients['coupling'], 1)
bending_condition = sp.Eq(rescaled_coefficients['bending'], 1)

coupling_condition


Eq(E*h*w0**2/v0, 1)

In [83]:
bending_condition

Eq(E**2*h**4*w0**2/v0**2, 1)

In [84]:
v0_scale = sp.solve(coupling_condition, v0)[0]
v0_scale

E*h*w0**2

In [85]:
bending_condition.subs(v0, v0_scale)

Eq(h**2/w0**2, 1)

In [86]:
w0_scale = sp.solve(bending_condition.subs(v0, v0_scale), w0)[1]
w0_scale


h

In [87]:
v0_scale = v0_scale.subs(w0, w0_scale)
v0_scale

E*h**3

In [89]:
coeff_list_scaled = {key: coeff.subs({v0: v0_scale, w0: w0_scale, x0: R}) 
                            for key, coeff in rescaled_coefficients.items()}
for key, value in coeff_list_scaled.items():
    latex_str = sp.latex(value)
    display(f"{key}", Math(f"{latex_str}"))

'membrane'

<IPython.core.display.Math object>

'bending'

<IPython.core.display.Math object>

'coupling'

<IPython.core.display.Math object>

'load_transverse'

<IPython.core.display.Math object>

'load_disclinations'

<IPython.core.display.Math object>

In [52]:
print(sp.latex(coeff_list_scaled['membrane'])), print(sp.latex(coeff_list_scaled['coupling'])), print(sp.latex(coeff_list_scaled['bending']))

1
1
1


(None, None, None)

In [53]:
print(sp.latex(coeff_list_scaled['load_transverse'])), print(sp.latex(coeff_list_scaled['load_disclinations']))

\frac{R^{4} p_{0}}{E h^{4}}
\frac{R^{2}}{h^{2}}


(None, None)

# Rescaled energy

$$
\tilde E(y):=
\underbrace{\frac{-1}{2}\int_\Omega |\nabla^2 v|^2 - \nu [v, v]}_{\text{membrane}} +
\underbrace{\frac{1}{2}c_\nu\int_\Omega |\Delta w|^2 - (1-\nu) [w, w]}_{\text{bending}} +
\underbrace{\frac{1}{2 }\int_\Omega J^T \nabla^2 v J: \nabla w\otimes \nabla w}_{\text{coupling}} 
- \frac{R^{4} p_{0}}{E h^{4}}\underbrace{\int_\Omega p w }_{\text{transv. pressure}} 
+ \frac{R^{2}}{h^{2}}\underbrace{ \sum_i\int_\Omega s_i \delta(x_i) v}_{\text{disclinations}} 
$$

where $\tilde E = E/\text{[membrane]}$,
$v,w$ are non-dimensional quantities, $c_\nu:=\frac{1}{12(1-\nu^2)}$.
Defining $\alpha:=\frac{R^{4} p_{0}}{E h^{4}}$, $\beta:=\frac{R^{2}}{h^{2}}$
one can always choose $E=R=1$
so that
$\alpha:=\tilde p_{0}$, $\beta:=\frac{1}{h^{2}}$



$$
\tilde E(y):=
\underbrace{\frac{-1}{2}\int_\Omega |\nabla^2 v|^2 - \nu [v, v]}_{\text{membrane}} +
\underbrace{\frac{1}{2}c_\nu\int_\Omega |\Delta w|^2 - (1-\nu) [w, w]}_{\text{bending}} +
\underbrace{\frac{1}{2 }\int_\Omega J^T \nabla^2 v J: \nabla w\otimes \nabla w}_{\text{coupling}} 
- \tilde p_{0}\underbrace{\int_\Omega p w }_{\text{transv. pressure}} 
+ \frac{1}{h^{2}}\underbrace{ \sum_i\int_\Omega s_i \delta(x_i) v}_{\text{disclinations}} 
$$


Graphene-like applications
$h\sim10^{-10} \text{m}, E\sim 10^{11}$

simulations: $R$ hundreds of atoms ($10^{-8}$ m)
large specimens: mm-size ($10^{-3}$ m)
 https://www.nature.com/articles/s41586-021-03753-3

$h/R \sim 10^{-2} \div 10^{-7}$

$(h/R)^4 \sim 10^{-8} \div 10^{-28}$

$E(h/R)^4 \sim 10^{3} \div 10^{-17}$

In [32]:
coeff_list_scaled['load_transverse']/coeff_list_scaled['load_disclinations']

p0/(E**2*h**3)

In [58]:
display(Math(sp.latex(v0_scale))), display(Math(sp.latex(w0_scale)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

(None, None)